# library

In [ ]:
import pandas as pd
import numpy as np
import requests
from io import StringIO

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# import data

In [ ]:
lien_csv = "https://raw.githubusercontent.com/pierrebeguin/Master_2_defi_2/main/data_defi_2.csv"
response = requests.get(lien_csv)
data = pd.read_csv(StringIO(response.text), sep=";")

# supprimer la 1er ligne
data.columns = data.iloc[0]
data = data.iloc[1:].reset_index(drop=True)

# renommer les colonnes
data = data.rename(columns={'Libellé.Prescription': 'medicament', 'Avis.Pharmaceutique' : 'avis'})

# remplacer le NA par ''
data['avis'] = data['avis'].fillna('')

display(data.head())

,medicament,avis,PLT
0,"CONTRAMAL 100 MG/ML, GOUTTES BUV (TRAMADOL)",30/12/16 pas d'indication,5.3
1,"IMOVANE 7.5 MG, CPR SÉCABLE (ZOPICLONE)","22/12/16 recommandé -> IMOVANE 3,75MG CP, 1 au...",4.1
2,"COLCHICINE 1 MG, CPR SÉCABLE (COLCHICINE)","au vue de la DFG, il est recommandé d'administ...",4.1
3,"PANTOPRAZOLE 40 MG, CPR GASTRO-RÉSISTANT (EUPA...",Dose curative et absence d'ATCD gastrique retr...,4.1
4,VANCOMYCINE 500 mg 1x/j,posologie infraT veuillez réévaluer la posolog...,3.1


# data_quest_1

In [ ]:
# Changer les classes '4','5','6.3' et '6.4' par '1' et le reste par '0'
# '1' -> erreur grave et '0' -> autre
data_quest_1 = data.copy()
data_quest_1['PLT'] = data_quest_1['PLT'].replace(['4.1', '4.2', '5.1', '5.2', '5.3', '6.3', '6.4'], 1)
data_quest_1['PLT'] = data_quest_1['PLT'].apply(lambda x: 0 if x != 1 else x)

#data_quest_1.head()

,medicament,avis,PLT
0,"CONTRAMAL 100 MG/ML, GOUTTES BUV (TRAMADOL)",30/12/16 pas d'indication,1
1,"IMOVANE 7.5 MG, CPR SÉCABLE (ZOPICLONE)","22/12/16 recommandé -> IMOVANE 3,75MG CP, 1 au...",1
2,"COLCHICINE 1 MG, CPR SÉCABLE (COLCHICINE)","au vue de la DFG, il est recommandé d'administ...",1
3,"PANTOPRAZOLE 40 MG, CPR GASTRO-RÉSISTANT (EUPA...",Dose curative et absence d'ATCD gastrique retr...,1
4,VANCOMYCINE 500 mg 1x/j,posologie infraT veuillez réévaluer la posolog...,0
...,...,...,...
23509,"EXFORGE 5 MG/160 MG, CPR",Substitué par amlodipine 5 mg 1 comprimé par j...,0
23510,"VELMETIA 50 MG/1 000 MG, CPR",Substitué par sitagliptine 50 mg 2 comprimés p...,0
23511,"ACTISKENAN 10 MG, GÉLULE (MORPHINE)","si prise systématique, proposons skenan LP en ...",0
23512,"ACTISKENAN 10 MG, GÉLULE (MORPHINE)","si prise systématique, ajouter skenan LP et la...",0


In [ ]:
data_quest_1.groupby('PLT').size()
#data_quest_1

,0
PLT,
0,18973
1,4541


In [ ]:
# définir X et y (features et labels)
X = data_quest_1['avis']
y = data_quest_1['PLT'].values

In [ ]:
# Tokenisation et vectorisation
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
sequences = tokenizer.texts_to_sequences(X)

In [ ]:
# Padding des séquences
max_length = max(len(seq) for seq in sequences)
X = pad_sequences(sequences, maxlen=max_length)

In [ ]:
# Diviser le jeu de données en ensemble d'entraînement et ensemble de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Modèle_1

In [ ]:
# créer un modèle
model_1 = Sequential()
model_1.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=8, input_length=max_length))
model_1.add(LSTM(16))  # Utilisation d'une couche LSTM
model_1.add(Dense(1, activation='sigmoid'))  # Couche de sortie binaire

# Compilation du modèle
model_1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

NameError: name 'tokenizer' is not defined

In [ ]:
# entraînement du modèle
model_1.fit(X_train, y_train, epochs=10, batch_size=1, validation_data=(X_test, y_test))

Epoch 1/10
18811/18811 ━━━━━━━━━━━━━━━━━━━━ 348s 18ms/step - accuracy: 0.8702 - loss: 0.3223 - val_accuracy: 0.9175 - val_loss: 0.2138
Epoch 2/10
 4564/18811 ━━━━━━━━━━━━━━━━━━━━ 4:19 18ms/step - accuracy: 0.9374 - loss: 0.1746


KeyboardInterrupt



# évaluation_1

In [ ]:
loss, accuracy = model_1.evaluate(X_test, y_test)
print(f"Perte : {loss}, Précision : {accuracy}")

In [ ]:
# Prédictions sur de nouvelles données
new_texts = ['Texte à prédire', 'Autre exemple']
new_sequences = tokenizer.texts_to_sequences(new_texts)
new_X = pad_sequences(new_sequences, maxlen=max_length)
predictions = model_1.predict(new_X)
print(predictions)

# test_data_quest_1

In [ ]:
lien_csv = "https://raw.githubusercontent.com/pierrebeguin/Master_2_defi_2/main/test_data_defi_2.csv"
response = requests.get(lien_csv)
data_test = pd.read_csv(StringIO(response.text), sep=";")

test_data_quest_1['PLT'] = test_data_quest_1['PLT'].replace(['4.1', '4.2', '5.1', '5.2', '5.3', '6.3', '6.4'], 1)
test_data_quest_1['PLT'] = test_data_quest_1['PLT'].apply(lambda x: 0 if x != 1 else x)

# remplacer le NA par ''
test_data_quest_1['avis'] = test_data_quest_1['avis'].fillna('')


display(test_data_test.head())

In [ ]:
y_pred = model_1.predict(test_data_quest_1)
y_pred = pd.Series(y_pred, name="Prédictions")
display(y_pred)
y_pred.value_counts()

# data_quest_2

In [ ]:
# ranger les classes
data_quest_2 = data.copy()
data_quest_2['PLT'] = pd.to_numeric(data_quest_2['PLT'], errors='coerce')
data_quest_2['PLT'] = np.floor(data_quest_2['PLT']).astype(int)


data_quest_2.head()

,medicament,avis,PLT
0,"CONTRAMAL 100 MG/ML, GOUTTES BUV (TRAMADOL)",30/12/16 pas d'indication,5
1,"IMOVANE 7.5 MG, CPR SÉCABLE (ZOPICLONE)","22/12/16 recommandé -> IMOVANE 3,75MG CP, 1 au...",4
2,"COLCHICINE 1 MG, CPR SÉCABLE (COLCHICINE)","au vue de la DFG, il est recommandé d'administ...",4
3,"PANTOPRAZOLE 40 MG, CPR GASTRO-RÉSISTANT (EUPA...",Dose curative et absence d'ATCD gastrique retr...,4
4,VANCOMYCINE 500 mg 1x/j,posologie infraT veuillez réévaluer la posolog...,3


In [ ]:
#data_quest_2.groupby('PLT').size()

In [ ]:
# définir X et y (features et labels)
X = data_quest_2['avis'].astype(str)
y = data_quest_2['PLT'].values

In [ ]:
# Tokenisation et vectorisation
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
sequences = tokenizer.texts_to_sequences(X)

In [ ]:
# Padding des séquences
max_length = max(len(seq) for seq in sequences)
X = pad_sequences(sequences, maxlen=max_length)

In [ ]:
# Diviser le jeu de données en ensemble d'entraînement et ensemble de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)